In [0]:
#find . -name '*.jpg' -exec mv {} ../Dataset/ \;

In [0]:
import numpy as np
import pandas as pd
import glob
from PIL import Image
from tqdm import tqdm
from keras.models import load_model
import matplotlib.pyplot as plt
# import numexpr as ne
import keras_metrics

Using TensorFlow backend.


In [0]:
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Flatten, Dense, Dropout
from keras.activations import sigmoid,relu
from keras.optimizers import Adam
from sklearn.metrics import classification_report,f1_score, confusion_matrix

In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [0]:
#Normalize
def normalize(arr):
    arr=arr.astype('float32')
    if arr.max() > 1.0:
        arr/=255.0
    return arr
#Grayscale
gray = lambda rgb : np.dot(rgb[... , :3] , [0.299 , 0.587, 0.114]) 

In [0]:
# #Postives
positive = glob.glob('./Postive/*.jpg')
y = np.array([normalize(gray(np.array(Image.open(fname).resize((64,64), Image.ANTIALIAS)))) for fname in tqdm(positive)])
finalp=[]
for i in range(0,len(y)):
    finalp.append(y[i].flatten())
del y
positive_df=pd.DataFrame({'X':finalp,'y':1})

100%|██████████| 2999/2999 [00:09<00:00, 317.54it/s]


In [0]:
# #Negatives
negative = glob.glob('./Negative/*.jpg')
n = np.array([normalize(gray(np.array(Image.open(fname).resize((64,64), Image.ANTIALIAS)))) for fname in tqdm(negative[:])])
final=[]
for i in range(0,len(n)):
    final.append(n[i].flatten())
negative_df=pd.DataFrame({'X':final,'y':0})

100%|██████████| 2776/2776 [00:37<00:00, 74.94it/s]


In [0]:
# negative_df = pd.read_csv('negatives.csv')
final_dataset=pd.concat([positive_df,negative_df])
final_dataset=final_dataset.sample(frac=1)
final_dataset.reset_index(inplace=True)
final_dataset.drop('index',inplace=True,axis=1)

In [0]:
final_dataset.head()

,X,y
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2,"[0.058529414, 0.07280784, 0.07376863, 0.085152...",0
3,"[0.855349, 0.85462356, 0.86594117, 0.8737843, ...",0
4,"[0.13361962, 0.102611765, 0.09141569, 0.086764...",0


In [0]:
another=[]
for i in range(len(final_dataset)):
    another.append(final_dataset.X[i].reshape(64,64,1))

#**Binary Face Classifier**

In [0]:
X=np.array(another)
y=final_dataset.y.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True, test_size=1/3, random_state=8613)
del model
model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(strides=(2,2)))
model.add(Dropout(0.05))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)
pred_train=np.round(model.predict(X_train))
pred_test=np.round(model.predict(X_test))
f1_score(y_test,pred_test)

Train on 3850 samples, validate on 1925 samples
Epoch 1/5
3850/3850 [==============================] - 2s 563us/step - loss: 0.3792 - acc: 0.8244 - val_loss: 0.3009 - val_acc: 0.8494
Epoch 2/5
3850/3850 [==============================] - 2s 432us/step - loss: 0.1396 - acc: 0.9558 - val_loss: 0.0944 - val_acc: 0.9714
Epoch 3/5
3850/3850 [==============================] - 2s 424us/step - loss: 0.0816 - acc: 0.9792 - val_loss: 0.0740 - val_acc: 0.9803
Epoch 4/5
3850/3850 [==============================] - 2s 419us/step - loss: 0.0524 - acc: 0.9857 - val_loss: 0.0504 - val_acc: 0.9901
Epoch 5/5
3850/3850 [==============================] - 2s 418us/step - loss: 0.0400 - acc: 0.9917 - val_loss: 0.0421 - val_acc: 0.9901


0.9904761904761905

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 30753     
Total params: 31,073
Trainable params: 31,073
Non-trainable params: 0
_________________________________________________________________


# **Transfer Learning**

In [0]:
X = pd.read_csv('data/X.csv',sep=' ',header=None, dtype=float)
y = pd.read_csv('data/y_williams_vs_others.csv',header=None)
y_williams=y.values.ravel()
X = X.as_matrix().reshape(13233,64,64,1)
X_train, X_test, y_train, y_test = train_test_split(X, y_williams,shuffle=True, test_size=1/3, random_state=8613,stratify=y_williams)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)
pred_train=np.round(model.predict(X_train))
pred_test=np.round(model.predict(X_test))
print('Train F1 Score ', f1_score(y_train, pred_train))
print('Test F1 Score ', f1_score(y_test, pred_test))

Train on 8822 samples, validate on 4411 samples
Epoch 1/20
8822/8822 [==============================] - 4s 440us/step - loss: 0.0657 - acc: 0.9905 - val_loss: 0.0251 - val_acc: 0.9961
Epoch 2/20
8822/8822 [==============================] - 4s 438us/step - loss: 0.0232 - acc: 0.9958 - val_loss: 0.0144 - val_acc: 0.9966
Epoch 3/20
8822/8822 [==============================] - 4s 438us/step - loss: 0.0157 - acc: 0.9960 - val_loss: 0.0116 - val_acc: 0.9971
Epoch 4/20
8822/8822 [==============================] - 4s 465us/step - loss: 0.0116 - acc: 0.9972 - val_loss: 0.0100 - val_acc: 0.9975
Epoch 5/20
8822/8822 [==============================] - 4s 455us/step - loss: 0.0102 - acc: 0.9976 - val_loss: 0.0097 - val_acc: 0.9975
Epoch 6/20
8822/8822 [==============================] - 4s 438us/step - loss: 0.0078 - acc: 0.9975 - val_loss: 0.0130 - val_acc: 0.9966
Epoch 7/20
8822/8822 [==============================] - 4s 441us/step - loss: 0.0074 - acc: 0.9982 - val_loss: 0.0094 - val_acc: 0.9977


In [0]:
model.save('initialized_model_williams_689.model')                                                                                                                                     